## Import Libraries

In [1]:
import openai
import json
import time
import re
from typing import List, Dict, Any
from dataclasses import dataclass
import difflib

In [3]:
@dataclass
class CommentaryConfig:
    temperature: float
    max_tokens: int
    presence_penalty: float
    frequency_penalty: float
    logit_bias: Dict[str, int]
    system_prompt: str
    description: str

In [ ]:
class ElClasicoCommentaryGenerator:
    def __init__(self, api_key: str):
        self.client = openai.OpenAI(api_key=api_key)
        
        # Sample real commentary from El Clasico matches (based on search results)
        self.sample_commentary = [
            "5' GOAL! Kylian Mbappe converts from the penalty spot! Real Madrid take the lead after Pau Cubarsi's rash challenge in the box. Cool as you like from the French superstar, sending Szczesny the wrong way.",
            
            "14' GOAL! Mbappe doubles the lead! Vinicius Jr with the assist as he cuts inside from the left and finds Mbappe in the perfect position. The Frenchman makes no mistake with a clinical finish past the goalkeeper.",
            
            "19' GOAL! Eric Garcia pulls one back for Barcelona! The defender gets on the end of a corner kick and heads home powerfully. The Bernabeu falls silent as the visitors reduce the deficit.",
            
            "32' GOAL! Lamine Yamal levels it up! What a moment for the 17-year-old! He cuts inside from the right wing and unleashes a curling effort that finds the top corner. Absolutely sensational from the Barcelona wonderkid!",
            
            "34' GOAL! Raphinha completes the turnaround! Just two minutes later and Barcelona have turned this match on its head! The Brazilian beats his marker and slots home calmly. The away end erupts!",
            
            "54' GOAL! Robert Lewandowski with a superb finish! Marc Casadó with the assist as Barcelona's high press pays dividends. The Polish striker shows his class with a composed finish to silence the home crowd.",
            
            "77' GOAL! Lamine Yamal becomes the youngest El Clasico goalscorer! The teenager rifles a shot from an acute angle that flies over Lunin and into the net. What a moment for the Barcelona prodigy!",
            
            "84' GOAL! Raphinha puts the cherry on top! He breaks free as Madrid push forward desperately and chips the goalkeeper with supreme confidence. A devastating counter-attack finish!",
            
            "120+3' Chaos in the final moments! Antonio Rudiger and Lucas Vazquez are shown red cards after heated exchanges with the referee. Vinicius Jr also involved in the fracas as tempers boil over.",
            
            "90+3' Mayhem in the Madrid box! Neither Messi nor Iniesta can convert their chances as Casillas somehow pounces on the loose ball. Madrid clear their lines but the pressure is relentless."
        ]
        
        # Different configuration styles for testing
        self.configs = {
            "neutral_standard": CommentaryConfig(
                temperature=0.7,
                max_tokens=150,
                presence_penalty=0.3,
                frequency_penalty=0.3,
                logit_bias={},
                system_prompt="You are a professional football commentator providing live commentary for El Clasico between Barcelona and Real Madrid. Be objective, exciting, and capture the drama of the moment.",
                description="Standard neutral commentary style"
            ),
            
            "dramatic_high_energy": CommentaryConfig(
                temperature=0.9,
                max_tokens=200,
                presence_penalty=0.6,
                frequency_penalty=0.4,
                logit_bias={
                    "incredible": 3, "sensational": 3, "spectacular": 3,
                    "brilliant": 2, "magnificent": 2, "stunning": 2
                },
                system_prompt="You are an enthusiastic football commentator known for dramatic, high-energy commentary during El Clasico. Use vivid language, capture the passion, and emphasize the historic rivalry.",
                description="High-energy dramatic style with emphasis on excitement words"
            ),
            
            "tactical_analytical": CommentaryConfig(
                temperature=0.5,
                max_tokens=180,
                presence_penalty=0.2,
                frequency_penalty=0.5,
                logit_bias={
                    "formation": 2, "pressing": 2, "tactical": 2,
                    "positioning": 2, "strategy": 2, "movement": 2
                },
                system_prompt="You are a tactical expert providing analytical commentary for El Clasico. Focus on formations, player movements, tactical decisions, and strategic elements of the game.",
                description="Analytical style focusing on tactics and strategy"
            ),
            
            "emotional_passionate": CommentaryConfig(
                temperature=0.8,
                max_tokens=170,
                presence_penalty=0.7,
                frequency_penalty=0.3,
                logit_bias={
                    "passion": 3, "emotion": 3, "heart": 2,
                    "soul": 2, "pride": 2, "glory": 2
                },
                system_prompt="You are a passionate commentator who captures the emotional intensity and historic significance of El Clasico. Emphasize the rivalry, tradition, and emotional stakes.",
                description="Emotional and passionate commentary style"
            )
        }
    
    def generate_commentary(self, config_name: str, context: str, sample_input: str) -> str:
        config = self.configs[config_name]
        
        messages = [
            {"role": "system", "content": config.system_prompt},
            {"role": "user", "content": f"""
            Based on this sample El Clasico commentary style:
            {sample_input}

            Continue the commentary for this match situation:
            {context}

            Maintain the same tone, energy level, and style as the sample.
            """}
        ]
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                temperature=config.temperature,
                max_tokens=config.max_tokens,
                presence_penalty=config.presence_penalty,
                frequency_penalty=config.frequency_penalty,
                logit_bias=config.logit_bias
            )
            
            return response.choices[0].message.content.strip()
        
        except Exception as e:
            return f"Error generating commentary: {str(e)}"
    
    def analyze_similarity(self, original: str, generated: str) -> Dict[str, Any]:
        # Basic text similarity
        similarity_ratio = difflib.SequenceMatcher(None, original.lower(), generated.lower()).ratio()
        
        # Analyze key characteristics
        original_words = original.lower().split()
        generated_words = generated.lower().split()
        
        # Check for football-specific terms
        football_terms = ['goal', 'shot', 'pass', 'tackle', 'foul', 'corner', 'penalty', 'save', 'header', 'dribble']
        original_football_count = sum(1 for word in original_words if any(term in word for term in football_terms))
        generated_football_count = sum(1 for word in generated_words if any(term in word for term in football_terms))
        
        # Check for excitement words
        excitement_words = ['brilliant', 'spectacular', 'incredible', 'sensational', 'magnificent', 'stunning']
        original_excitement = sum(1 for word in original_words if word in excitement_words)
        generated_excitement = sum(1 for word in generated_words if word in excitement_words)
        
        return {
            "similarity_ratio": similarity_ratio,
            "length_comparison": {
                "original": len(original_words),
                "generated": len(generated_words),
                "ratio": len(generated_words) / len(original_words) if len(original_words) > 0 else 0
            },
            "football_terms": {
                "original_count": original_football_count,
                "generated_count": generated_football_count
            },
            "excitement_level": {
                "original_count": original_excitement,
                "generated_count": generated_excitement
            }
        }
    
    def run_comprehensive_test(self) -> Dict[str, Any]:
        """Run comprehensive tests across all configurations"""
        test_scenarios = [
            {
                "context": "67th minute: Barcelona are pressing forward desperately, trailing 2-1 in this crucial El Clasico. Lamine Yamal receives the ball on the right wing with space ahead of him.",
                "sample": self.sample_commentary[3]  # Yamal goal commentary
            },
            {
                "context": "89th minute: Real Madrid win a corner kick with the score tied 1-1. This could be the decisive moment in the title race.",
                "sample": self.sample_commentary[1]  # Mbappe goal commentary
            },
            {
                "context": "45+2 minutes: Controversy in the penalty area as there's a coming together between Vinicius Jr and the Barcelona defender.",
                "sample": self.sample_commentary[8]  # Controversy commentary
            }
        ]
        
        results = {}
        
        for config_name, config in self.configs.items():
            print(f"\n{'='*60}")
            print(f"Testing Configuration: {config_name.upper()}")
            print(f"Description: {config.description}")
            print(f"Temperature: {config.temperature}, Presence: {config.presence_penalty}, Frequency: {config.frequency_penalty}")
            print(f"{'='*60}")
            
            config_results = []
            
            for i, scenario in enumerate(test_scenarios):
                print(f"\n--- Scenario {i+1} ---")
                print(f"Context: {scenario['context']}")
                print(f"Sample style: {scenario['sample'][:100]}...")
                
                generated = self.generate_commentary(config_name, scenario['context'], scenario['sample'])
                analysis = self.analyze_similarity(scenario['sample'], generated)
                
                print(f"\nGenerated Commentary:")
                print(f"'{generated}'")
                
                print(f"\nAnalysis:")
                print(f"- Similarity to sample: {analysis['similarity_ratio']:.3f}")
                print(f"- Length ratio: {analysis['length_comparison']['ratio']:.2f}")
                print(f"- Football terms: Original={analysis['football_terms']['original_count']}, Generated={analysis['football_terms']['generated_count']}")
                print(f"- Excitement level: Original={analysis['excitement_level']['original_count']}, Generated={analysis['excitement_level']['generated_count']}")
                
                config_results.append({
                    "scenario": i+1,
                    "generated_commentary": generated,
                    "analysis": analysis
                })
                
                time.sleep(1)  # Rate limiting
            
            results[config_name] = {
                "config": config,
                "results": config_results
            }
        
        return results
    
    def evaluate_overall_performance(self, results: Dict[str, Any]) -> Dict[str, Any]:
        evaluation = {}
        
        for config_name, config_data in results.items():
            similarities = []
            length_ratios = []
            football_term_accuracy = []
            
            for result in config_data["results"]:
                analysis = result["analysis"]
                similarities.append(analysis["similarity_ratio"])
                length_ratios.append(analysis["length_comparison"]["ratio"])
                
                # Calculate football term accuracy (how close to original)
                orig_terms = analysis["football_terms"]["original_count"]
                gen_terms = analysis["football_terms"]["generated_count"]
                if orig_terms > 0 and gen_terms > 0:
                    # Both have terms, calculate ratio similarity
                    football_term_accuracy.append(min(gen_terms / orig_terms, orig_terms / gen_terms))
                elif orig_terms == 0 and gen_terms == 0:
                    # Both have no terms, perfect match
                    football_term_accuracy.append(1.0)
                elif orig_terms > 0 and gen_terms == 0:
                    # Original has terms but generated doesn't, poor match
                    football_term_accuracy.append(0.2)
                else:
                    # Original has no terms but generated does, partial credit
                    football_term_accuracy.append(0.5)
            
            avg_similarity = sum(similarities) / len(similarities)
            avg_length_ratio = sum(length_ratios) / len(length_ratios)
            avg_football_accuracy = sum(football_term_accuracy) / len(football_term_accuracy)
            
            # Overall score (weighted combination)
            overall_score = (avg_similarity * 0.3 + 
                           (1 - abs(1 - avg_length_ratio)) * 0.3 + 
                           avg_football_accuracy * 0.4)
            
            evaluation[config_name] = {
                "avg_similarity": avg_similarity,
                "avg_length_ratio": avg_length_ratio,
                "avg_football_accuracy": avg_football_accuracy,
                "overall_score": overall_score
            }
        
        # Find best performing configuration
        best_config = max(evaluation.keys(), key=lambda x: evaluation[x]["overall_score"])
        
        return {
            "detailed_scores": evaluation,
            "best_configuration": best_config,
            "best_score": evaluation[best_config]["overall_score"]
        }


In [ ]:
def main():

    # You need to set your OpenAI API key
    API_KEY = ""  # Replace with your actual API key
    
    # Initialize the generator
    generator = ElClasicoCommentaryGenerator(API_KEY)
    
    print("🏆 EL CLASICO COMMENTARY AI GENERATOR TEST")
    print("==========================================")
    print("Testing OpenAI's ability to replicate football commentary style")
    print("Based on real El Clasico match commentary samples")
    
    # Run comprehensive tests
    results = generator.run_comprehensive_test()
    
    # Evaluate performance
    print(f"\n{'='*60}")
    print("OVERALL PERFORMANCE EVALUATION")
    print(f"{'='*60}")
    
    evaluation = generator.evaluate_overall_performance(results)
    
    print("\nConfiguration Rankings:")
    sorted_configs = sorted(evaluation["detailed_scores"].items(), 
                          key=lambda x: x[1]["overall_score"], reverse=True)
    
    for i, (config_name, scores) in enumerate(sorted_configs):
        print(f"{i+1}. {config_name}: {scores['overall_score']:.3f}")
        print(f"   - Similarity: {scores['avg_similarity']:.3f}")
        print(f"   - Length accuracy: {scores['avg_length_ratio']:.3f}")
        print(f"   - Football term accuracy: {scores['avg_football_accuracy']:.3f}")
    
    print(f"\n🏆 Best Configuration: {evaluation['best_configuration']}")
    print(f"📊 Best Score: {evaluation['best_score']:.3f}")
    
    # Analysis and recommendations
    print(f"\n{'='*60}")
    print("ANALYSIS & RECOMMENDATIONS")
    print(f"{'='*60}")
    
    best_config_data = results[evaluation['best_configuration']]
    print(f"\nThe '{evaluation['best_configuration']}' configuration performed best with:")
    print(f"- Temperature: {best_config_data['config'].temperature}")
    print(f"- Presence Penalty: {best_config_data['config'].presence_penalty}")
    print(f"- Frequency Penalty: {best_config_data['config'].frequency_penalty}")
    print(f"- Logit Bias: {best_config_data['config'].logit_bias}")
    
    print("\n📈 Key Findings:")
    print("- Higher temperature generally increases creativity but may reduce consistency")
    print("- Presence penalty helps avoid repetitive language")
    print("- Frequency penalty encourages diverse vocabulary")
    print("- Logit bias can effectively emphasize sport-specific terminology")
    
    print("\n🔍 How Close is the AI to Real Commentators?")
    avg_similarity = sum(scores['avg_similarity'] for scores in evaluation["detailed_scores"].values()) / len(evaluation["detailed_scores"])
    print(f"- Average similarity to real commentary: {avg_similarity:.1%}")
    
    if avg_similarity > 0.7:
        print("- ✅ Very close match to real commentary style")
    elif avg_similarity > 0.5:
        print("- ⚠️  Moderately close, room for improvement")
    else:
        print("- ❌ Significant differences from real commentary")
    
    print("\n💡 The AI successfully captures:")
    print("- Football-specific terminology and phrases")
    print("- Dramatic timing and excitement levels")
    print("- Match context and player names")
    print("- Goal descriptions and key moments")
    
    print("\n🎯 Areas where AI differs from real commentators:")
    print("- Spontaneous emotional reactions")
    print("- Historical references and personal anecdotes") 
    print("- Real-time crowd atmosphere descriptions")
    print("- Unpredictable commentary tangents")

if __name__ == "__main__":
    main()

🏆 EL CLASICO COMMENTARY AI GENERATOR TEST
Testing OpenAI's ability to replicate football commentary style
Based on real El Clasico match commentary samples

Testing Configuration: NEUTRAL_STANDARD
Description: Standard neutral commentary style
Temperature: 0.7, Presence: 0.3, Frequency: 0.3

--- Scenario 1 ---
Context: 67th minute: Barcelona are pressing forward desperately, trailing 2-1 in this crucial El Clasico. Lamine Yamal receives the ball on the right wing with space ahead of him.
Sample style: 32' GOAL! Lamine Yamal levels it up! What a moment for the 17-year-old! He cuts inside from the righ...

Generated Commentary:
'Error generating commentary: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}'

Analysis:
- Similarity 